# **Step1_AI면접관 Agent v1.0**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 라이브러리

In [ ]:
!pip install -r /content/drive/MyDrive/project_genai/requirements.txt -q

### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY
    * NGROK_AUTHTOKEN

In [ ]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/project_genai/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [ ]:
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-r111drRrWBH3MHbjiUfFop


## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

In [ ]:
%%writefile app.py

## 1. 라이브러리 로딩 ---------------------------------------------
import pandas as pd
import numpy as np
import os
import openai
import random
import ast
import fitz
from docx import Document

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict, List, Dict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.output_parsers import CommaSeparatedListOutputParser
from langgraph.graph import StateGraph, END

## ---------------- 1단계 : 사전준비 ----------------------

# 1) 파일 입력 --------------------
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

# 2) State 선언 --------------------
from typing import TypedDict, List, Dict

class ExamState(TypedDict):
    # 고정 정보
    exam_text: str
    exam_summary: str
    exam_keywords: List[str]
    question_strategy: Dict[str, Dict]

    # 문답 로그
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation : List[Dict[str, str]]
    next_step : str

# 3) resume 분석 --------------------
def analyze_exam(state: ExamState) -> ExamState:
    exam_text = state.get("exam_text", "")
    if not exam_text:
        raise ValueError("exam_text가 비어 있습니다. 먼저 텍스트를 추출해야 합니다.")

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 요약 프롬프트 구성
    summary_prompt = ChatPromptTemplate.from_template(
        '''당신은 고등학교 사회탐구 과목인 '정치와 법' 기출문제를 바탕으로 새로운 모의고사를 설계하는 AI입니다.
        다음 기출문제 내용에서 새로운 문제를 만들기 위해 필요한 내용을 문제별로 모두 요약 해줘. 총 20개의 요약이 나와야해(요약시 ** 기호는 사용하지 말것):\n\n{exam_text}'''
    )
    formatted_summary_prompt = summary_prompt.format(exam_text=exam_text)
    summary_response = llm.invoke(formatted_summary_prompt)
    exam_summary = summary_response.content.strip()

    # 키워드 추출 프롬프트 구성
    keyword_prompt = ChatPromptTemplate.from_template(
        '''당신은 고등학교 사회탐구 과목인 '정치와 법' 기출문제를 바탕으로 새로운 모의고사를 설계하는 AI입니다.
        다음 기출문제 내용에서 새로운 문제를 만들기 위한 중요한 핵심 키워드를 문제당 1개씩 총 20개 추출해줘.
        키워드는 그 문제에서 물어보는 고등학교 수능 개념을 추출해줘.
        도출한 핵심 키워드만 쉼표로 구분해줘:\n\n{exam_text}'''
    )
    formatted_keyword_prompt = keyword_prompt.format(exam_text=exam_text)
    keyword_response = llm.invoke(formatted_keyword_prompt)

    parser = CommaSeparatedListOutputParser()
    exam_keywords = parser.parse(keyword_response.content)

    return {
        **state,
        "exam_summary": exam_summary,
        "exam_keywords": exam_keywords,
    }

# 4) 질문 전략 수립 --------------------
import json

def generate_question_strategy(state: ExamState) -> ExamState:
    exam_summary = state.get("exam_summary", "")
    exam_keywords = ", ".join(state.get("exam_keywords", []))

    prompt = ChatPromptTemplate.from_template("""
    당신은 고등학교 사회탐구 과목인 '정치와 법' 기출문제를 바탕으로 **수능형 사례 기반 5지선다형 모의고사**를 설계하는 AI입니다.

    - 기출문제 요약:
    {exam_summary}

    - 기출문제 키워드:
    {exam_keywords}

      출제 조건:
    - 문제는 반드시 2025학년도 기준 고등학교 '정치와 법' 교과서 및 수능·모의고사 범위 내에서만 출제하세요.
    - 문제의 형식, 선택지 구성과 문제를 푸는 데 필요한 개념은 다음을 참고하되, 사례는 새롭게 구성하세요.
    - 각 문제는 다음과 같은 구조여야 합니다:
  {{
    "문제번호": 1,
    "문제": "다음 사례에 대한 법적 판단으로 옳은 것은?",
    "사례": "갑은 을과 혼인 후 A를 낳고 살다가 생활필수품 구매를 위해 친구 병으로부터 여러 차례 돈을 빌려 사용할 만큼 경제적 문제가 지속되자 결국 이혼 숙려 기간을 거쳐 이혼하였다. 한편 정과 무는 B와 C를 낳고 살았으나 무의 부정한 행위로 이혼하였으며 B는 정과, C는 무와 살기로 결정하였다. 몇 년 뒤, A를 홀로 양육하던 갑은 정과 혼인하였으며 정은 A를 친양자로 입양하였다. 이후 무가 갑작스런 사고로 사망하면서 갑은 B와 C를 친양자로 입양하였다."",
    "선택지": ["① 갑과의 혼인 기간 동안 을은 갑이 병에게 빌린 생활필수품 구매 비용을 갚을 의무가 없다.",
     "② 갑과 을의 이혼은 가정 법원에서 이혼 의사를 확인받는 즉시 그 효력이 발생한다.",
      "③ 무는 정과의 이혼 시 혼인 중 공동으로 마련한 재산에 대해 재산 분할을 청구할 수 없다.",
       "④ 정이 A를 입양함에 따라 을과 A의 친자 관계는 종료된다.",
        "⑤ 입양으로 인해 B, C 모두 갑과 정의 혼인 외 출생자로 간주된다."],
    "정답": "④"
  }}
        "사례" : "
    - 총 20개의 문제를 생성하고, 출력은 문제 리스트만 포함된 JSON 배열 형태로 출력하세요.

       출력 형식 예시 (JSON 리스트만 출력):

    [
      {{
        "문제1": "다음 사례에 대한 법적 판단으로 옳은 것은?"
        "사례" : "갑은 을과 혼인 후 A를 낳고 살다가 생활필수품 구매를 위해 친구 병으로부터 여러 차례 돈을 빌려 사용할 만큼 경제적 문제가 지속되자 결국 이혼 숙려 기간을 거쳐 이혼하였다. 한편 정과 무는 B와 C를 낳고 살았으나 무의 부정한 행위로 이혼하였으며 B는 정과, C는 무와 살기로 결정하였다. 몇 년 뒤, A를 홀로 양육하던 갑은 정과 혼인하였으며 정은 A를 친양자로 입양하였다. 이후 무가 갑작스런 사고로 사망하면서 갑은 B와 C를 친양자로 입양하였다."
        "선택지": [
          "① 갑과의 혼인 기간 동안 을은 갑이 병에게 빌린 생활필수품 구매 비용을 갚을 의무가 없다.",
          "② 갑과 을의 이혼은 가정 법원에서 이혼 의사를 확인받는 즉시 그 효력이 발생한다.",
          "③ 무는 정과의 이혼 시 혼인 중 공동으로 마련한 재산에 대해 재산 분할을 청구할 수 없다.",
          "④ 정이 A를 입양함에 따라 을과 A의 친자 관계는 종료된다.",
          "⑤ 입양으로 인해 B, C 모두 갑과 정의 혼인 외 출생자로 간주된다."
        ],
        "정답": "④"
      }},
      ...
    ]

    ❗ 출력은 코드블럭 없이 반드시 순수 JSON 문자열만 출력해야 합니다.
    ❗ 모든 키와 값은 큰따옴표(")로 감싸야 하며, 정답은 반드시 "①", "②" 등의 문자열로 표시합니다.
    """)

    llm = ChatOpenAI(model="gpt-4o-mini")
    formatted_prompt = prompt.format(
        exam_summary=exam_summary,
        exam_keywords=exam_keywords
    )
    response = llm.invoke(formatted_prompt)
    raw_output = response.content.strip()

    # 코드블럭 제거
    if "```" in raw_output:
        import re
        raw_output = re.sub(r"```(?:json)?", "", raw_output).replace("```", "").strip()

    # JSON 파싱
    try:
        question_list = json.loads(raw_output)
    except json.JSONDecodeError as e:
        raise ValueError("문제를 JSON으로 변환하는 데 실패했습니다.\n원본:\n" + raw_output) from e

    return {
        **state,
        "question_strategy": question_list  # 리스트 직접 저장
    }



# 5) 1단계 하나로 묶기 --------------------

def preProcessing_Exam(file_path: str) -> ExamState:
    # 파일에서 텍스트 추출
    exam_text = extract_text_from_file(file_path)

    # state 초기화
    initial_state: ExamState = {
        "exam_text": exam_text,
        "exam_summary": '',
        "exam_keywords": [],
        "question_strategy": [],
        "current_question": '',
        "current_answer": '',
        "current_strategy": '',
        "conversation": [],
        "evaluation": [],
        "next_step": ''
    }

    # exam 분석 → 요약 및 키워드 생성 (예: LLM 요약)
    state = analyze_exam(initial_state)

    # 문제 리스트 생성
    state = generate_question_strategy(state)

    # 문제 리스트
    question_list = state["question_strategy"]

    # 하나 선택
    selected_question = random.choice(question_list)

    return {
        **state,
        "current_question": selected_question,
        "current_strategy": ""  # 주제 구분 없음
    }



## ---------------- 2단계 : 면접 Agent ----------------------

import random
import re
from typing import Dict, Any

ExamState = Dict[str, Any]  # 상태를 저장하는 타입

# 사용자 답변을 상태에 저장
def update_current_answer(state: ExamState, user_answer: str) -> ExamState:
    return {
        **state,
        "current_answer": user_answer.strip()
    }

# 정답 여부만 판단하는 평가 함수
def evaluate_answer(state: ExamState) -> ExamState:
    current_question = state.get("current_question", {})
    current_answer = state.get("current_answer", "").strip()

    # 정답
    correct_answer = current_question.get("정답", "").strip()

    # 사용자 입력이 정답과 일치하는지 확인
    result = {
        "정답여부": "맞음" if current_answer == correct_answer else "틀림",
        "정답": correct_answer
    }

    # 대화 저장
    state["conversation"].append({
        "question": current_question,
        "answer": current_answer
    })

    # 평가 저장
    evaluation = state.get("evaluation", [])
    result["question_index"] = len(state["conversation"]) - 1
    evaluation.append(result)

    return {
        **state,
        "evaluation": evaluation
    }



# 3) 문제풀이 진행 검토 --------------------
def decide_next_step(state: ExamState) -> ExamState:
    conversation = state.get("conversation", [])

    # (1) 총 질문이 20개를 초과하면 종료
    if len(conversation) >= 20:
        next_step = "end"

    # (2) 아직 문제 남아있으면 추가 질문
    else:
        next_step = "additional_question"

    return {
        **state,
        "next_step": next_step
    }

# 4) 해설 생성 --------------------
def generate_explanation(state: ExamState) -> ExamState:
    llm = ChatOpenAI(model="gpt-4o-mini")
    current_question = state.get("current_question", {})
    question_text = current_question.get("문제", "")
    choices = "\n".join(current_question.get("선택지", []))
    correct_answer = current_question.get("정답", "")

    prompt = ChatPromptTemplate.from_template("""
당신은 '정치와 법' 과목 수능형 객관식 문제에 대한 해설을 작성하는 AI입니다.

아래는 문제와 선택지, 정답입니다:
- 문제: {question_text}
- 선택지:
{choices}
- 정답: {correct_answer}

이 문제에 대한 **해설(풀이 과정)**을 작성하세요.
학생이 이해할 수 있도록 문제를 어떤 법적 개념과 판례를 통해 해결해야 하는지 구체적으로 설명하세요.
답의 근거뿐 아니라, 오답이 왜 틀렸는지도 간략히 설명하세요.

형식: 한 문단으로 구성하며, 학생이 공부에 활용할 수 있도록 친절하고 명확하게 작성하세요.
""")

    formatted_prompt = prompt.format(
        question_text=question_text,
        choices=choices,
        correct_answer=correct_answer
    )

    response = llm.invoke(formatted_prompt)
    explanation = response.content.strip()

    # 해설 저장
    current_question["해설"] = explanation

    # 사용한 문제 제외하고 새로운 문제 선택
    used_questions = [turn["question"] for turn in state["conversation"]]
    remaining_questions = [q for q in state["question_strategy"] if q not in used_questions]

    if remaining_questions:
        next_question = random.choice(remaining_questions)
    else:
        next_question = {"문제": "모든 문제가 출제되었습니다.", "선택지": [], "사례": ""}

    return {
        **state,
        "current_question": next_question,
        "current_answer": ""
    }


# 5) 문제풀이 피드백 보고서 --------------------
def summarize_exam(state: ExamState) -> ExamState:
    summary_blocks = []
    print("\n\U0001F4D8 문제풀이 종료 보고서: 문제 풀이 및 정답 확인")
    print("=" * 70)

    for i, turn in enumerate(state["conversation"]):
        qnum = i + 1
        question = turn["question"].get("문제", "").strip()
        choices = turn["question"].get("선택지", [])
        answer = turn["answer"]
        eval_result = state["evaluation"][i] if i < len(state["evaluation"]) else {}
        correct = eval_result.get("정답", "")
        result = eval_result.get("정답여부", "")
        explanation = turn["question"].get("해설", "해설이 생성되지 않았습니다.")

        # 콘솔 출력
        print(f"[문제 {qnum}] {question}")
        print("[선택지]")
        for choice in choices:
            print(f"   {choice}")
        print(f"[당신의 답변] {answer}")
        print(f"[정답] {correct}")
        print(f"[정답 여부] {result}")
        print(f"[해설] {explanation}")
        print("-" * 70)

        # UI용 블록 추가
        block = f"""### 문제 {qnum}
**문제:** {question}
**내 답:** {answer} / **정답:** {correct} ({result})
**해설:** {explanation}\n"""
        summary_blocks.append(block)

        # 해설이 없었다면 생성 (보완)
        if "해설" not in turn["question"]:
            state["current_question"] = turn["question"]
            state = generate_explanation(state)
            turn["question"] = state["current_question"]

    # 모든 요약 텍스트를 하나로 합쳐서 반환용 추가
    state["final_summary"] = "\n\n".join(summary_blocks)
    return state

# 6) Agent --------------------
# 분기 판단 함수
def route_next(state: ExamState) -> Literal["generate", "summarize"]:
    return "summarize" if state["next_step"] == "end" else "generate"

# 그래프 정의 시작
builder = StateGraph(ExamState)

# 노드 추가
builder.add_node("evaluate", evaluate_answer)
builder.add_node("decide", decide_next_step)
builder.add_node("generate", generate_explanation)
builder.add_node("summarize", summarize_exam)

# 노드 연결
builder.set_entry_point("evaluate")
builder.add_edge("evaluate", "decide")
builder.add_conditional_edges("decide", route_next)
builder.add_edge("generate", END)      # 루프
builder.add_edge("summarize", END)            # 종료

# 컴파일
graph = builder.compile()
#-------------------------------------------------------------------


import gradio as gr

# 세션 상태 초기화 함수
def initialize_state():
    return {
        "state": None,
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

# 파일 업로드 후 문제풀이 초기화
def upload_and_initialize(file_obj, session_state):
    if file_obj is None:
        return session_state, [["🤖 AI", "📂 파일을 업로드해주세요."]]

    file_path = file_obj.name
    state = preProcessing_Exam(file_path)

    session_state["state"] = state
    session_state["interview_started"] = True

    current = state["current_question"]
    question_text = current.get("문제", "")
    case_text = current.get("사례", "")
    choices_text = "\n".join(current.get("선택지", []))
    full_question = f"{case_text}\n\n{question_text}\n{choices_text}"

    chat_history = [
        ["🤖 AI", "문제를 생성 중입니다..."],
        ["🤖 AI 문제", full_question]
    ]
    session_state["chat_history"] = chat_history
    full_question = f"[문제 {current.get('문제번호', '?')}] {case_text}\n\n{question_text}\n{choices_text}"

    return session_state, chat_history

# 답변 처리 및 다음 문제 생성
def chat_interview(user_input, session_state):
    if not session_state["interview_started"]:
        return session_state, [["🤖 AI", "먼저 기출문제를 업로드하고 시작하세요."]]

    # 사용자 답변 저장
    session_state["chat_history"].append(["🙋‍♂️ 사용자", user_input])
    session_state["state"] = update_current_answer(session_state["state"], user_input)

    # 상태 업데이트
    session_state["state"] = graph.invoke(session_state["state"])

    if session_state["state"]["next_step"] == "end":
        session_state["interview_ended"] = True
        session_state["state"] = summarize_exam(session_state["state"])

        summary_text = session_state["state"].get("final_summary", "결과 요약 없음")
        session_state["chat_history"].append(["🤖 AI 문제", "✅ 문제풀이가 종료되었습니다.\n\n" + summary_text])

        return session_state, session_state["chat_history"]
    else:
        current = session_state["state"]["current_question"]
        question_text = current.get("문제", "")
        case_text = current.get("사례", "")
        choices_text = "\n".join(current.get("선택지", []))
        qnum = current.get("문제번호", "?")
        full_question = f"[문제 {qnum}] {case_text}\n\n{question_text}\n{choices_text}"

        session_state["chat_history"].append(["🤖 AI 문제", full_question])
        return session_state, session_state["chat_history"]


# Gradio UI 구성
with gr.Blocks() as demo:
    session_state = gr.State(initialize_state())

    gr.Markdown("# 🧠 AI 문제풀이 시스템\n기출문제를 업로드하고 수능형 문제를 풀어보세요!")

    with gr.Row():
        file_input = gr.File(label="기출문제 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("문제풀이 시작")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(show_label=False, placeholder="답을 ①, ②, ③ 등의 형식으로 입력 후 Enter를 누르세요.")

    upload_btn.click(upload_and_initialize, inputs=[file_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(chat_interview, inputs=[user_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(lambda: "", None, user_input)

# 실행
demo.launch(share=True)


Overwriting app.py


## **3. 실행**

In [ ]:
!python app.py

/content/app.py:485: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://0aa6fa9fd96dd72236.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 3075, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
 